In [ ]:
import numpy as np
from tqdm import tqdm
from pathlib import Path

from lac.slam.semantic_feature_tracker import SemanticFeatureTracker
from lac.slam.frontend import Frontend
from lac.slam.backend import Backend
from lac.utils.plotting import plot_poses, plot_loop_closures
from lac.util import load_data, load_stereo_images, positions_rmse_from_poses, rotation_matrix_error

%load_ext autoreload
%autoreload 2

# Load data


In [ ]:
# Load the data logs
data_path = "/home/shared/data_raw/LAC/runs/triangles_preset6"
# data_path = "../../../output/NavAgent/triangles_preset7_bad"
initial_pose, lander_pose, poses, imu_data, cam_config = load_data(data_path)
print(f"Loaded {len(poses)} poses")

In [ ]:
# images = load_images(data_path, cameras=["FrontLeft", "FrontRight"], start_frame=0, end_frame=10000)
left_imgs, right_imgs = load_stereo_images(data_path, start_frame=0)
images = {"FrontLeft": left_imgs, "FrontRight": right_imgs}

# Initialize


In [ ]:
START_FRAME = 80
END_FRAME = len(poses) - 1

# TODO: we should probably keyframe the first frame (give it to the backend)

feature_tracker = SemanticFeatureTracker(cam_config)
frontend = Frontend(feature_tracker, initial_pose=initial_pose)
backend = Backend(poses[START_FRAME], feature_tracker)

init_data = {
    "step": START_FRAME,
    "FrontLeft": left_imgs[START_FRAME],
    "FrontRight": right_imgs[START_FRAME],
    "imu": imu_data[START_FRAME],
}

frontend.initialize(init_data)

# Run


In [ ]:
eval_poses = [poses[START_FRAME]]

for frame in tqdm(range(START_FRAME + 2, END_FRAME, 2)):
    data = {
        "step": frame,
        "FrontLeft": left_imgs[frame],
        "FrontRight": right_imgs[frame],
        "imu": imu_data[frame],
    }
    data = frontend.process_frame(data)
    backend.update(data)
    eval_poses.append(poses[frame])

# Evaluate localization


In [ ]:
trajectory = backend.get_trajectory()

In [ ]:
fig = plot_poses(eval_poses, no_axes=True, color="black", name="Ground truth")
fig = plot_poses(trajectory, fig=fig, no_axes=True, color="orange", name="Backend poses")
fig = plot_loop_closures(trajectory, backend.loop_closures, fig=fig, showlegend=False)
fig.update_layout(height=900, width=1600, scene_aspectmode="data")
fig.show()

In [ ]:
print(f"RMSE: {positions_rmse_from_poses(eval_poses, trajectory)}")

## Evaluate loop closures


In [ ]:
from lac.slam.loop_closure import estimate_loop_closure_pose
from lac.util import rotations_rmse_from_poses

In [ ]:
backend.loop_closures

In [ ]:
frame1 = 20
frame2 = 2600
img1 = left_imgs[frame1]
img2 = left_imgs[frame2]

print(f"inter-frame distance: {np.linalg.norm(poses[frame1][:3, 3] - poses[frame2][:3, 3])}")
print(f"inter-frame angle: {rotation_matrix_error(poses[frame1][:3, :3], poses[frame2][:3, :3])}")

# Estimate relative pose
rel_pose = estimate_loop_closure_pose(
    feature_tracker,
    left_imgs[frame1],
    right_imgs[frame1],
    left_imgs[frame2],
    right_imgs[frame2],
)
true_rel_pose = np.linalg.inv(poses[frame1]) @ poses[frame2]
print(f"rotation RMSE: {rotations_rmse_from_poses([rel_pose], [true_rel_pose])}")
print(f"translation RMSE: {positions_rmse_from_poses([rel_pose], [true_rel_pose])}")

# Evaluate mapping


In [ ]:
from lac.mapping.mapper import process_map
from lac.mapping.map_utils import get_geometric_score, get_rocks_score
from lac.utils.plotting import plot_heightmaps, plot_rock_maps, plot_rock_results, plot_height_error

In [ ]:
point_map = backend.project_point_map()

In [ ]:
point_map.save(Path(data_path) / "semantic_points_runslam.npz")

In [ ]:
# ground_truth_map = np.load(
#     "/home/shared/data_raw/LAC/heightmaps/competition/Moon_Map_01_preset_4.dat",
#     allow_pickle=True,
# )
# ground_truth_map = np.load(Path(data_path) / "Moon_Map_01_6_rep0.dat", allow_pickle=True)
ground_truth_map = np.load("../../../results/Moon_Map_01_7_rep0.dat", allow_pickle=True)

In [ ]:
agent_map = ground_truth_map.copy()
agent_map = process_map(point_map, agent_map)

In [ ]:
print(f"Geometric score: {get_geometric_score(ground_truth_map, agent_map)}")
print(f"Rocks score: {get_rocks_score(ground_truth_map, agent_map)}")

In [ ]:
plot_rock_results(ground_truth_map, agent_map)

In [ ]:
plot_height_error(ground_truth_map, agent_map)

In [ ]:
plot_heightmaps(ground_truth_map, agent_map)

In [ ]:
plot_rock_maps(ground_truth_map, agent_map)